In [1]:
import pandas as pd
import os
import numpy as np
import scipy.stats as stats

pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

In [2]:
PATH = 'ml/results_rf_290322.csv'

In [3]:
results = pd.read_csv(PATH, index_col=0)

chunk_sizes = results.chunk_size.unique()

complete = results[results.restriction == 'complete']
complete = complete.dropna(how='all', axis=1)
grouped_complete = complete.groupby(by=['restriction', 'modality', 'chunk_size'])
grouped_complete[['mcc', 'auc']].median()

mcc       auc
restriction modality      chunk_size                    
complete    morphological 0.0         0.943950  0.995410
                          25.0        0.970074  0.999426
                          50.0        0.970074  0.999426
                          100.0       0.970074  0.999426
                          200.0       0.970074  0.999426
                          400.0       0.970074  0.999426
                          800.0       0.970074  0.998853
                          1600.0      0.970074  0.998853
            spatial       0.0         0.494662  0.827883
                          25.0        0.614339  0.920539
                          50.0        0.578518  0.901320
                          100.0       0.578518  0.885255
                          200.0       0.559628  0.876363
                          400.0       0.554939  0.874068
                          800.0       0.539955  0.869191
                          1600.0      0.492887  0.859151
            temporal      0.0         0.814973  0.975043
                          25.0        0.749261  0.975330
                          50.0        0.782901  0.979633
                          100.0       0.816169  0.980207
                          200.0       0.816169  0.980207
                          400.0       0.846215  0.980780
                          800.0       0.847711  0.982501
                          1600.0      0.820998  0.981641

## WF

In [4]:
wf_res = complete[complete.modality == 'morphological']
grouped_wf = wf_res.groupby(by=['chunk_size'])
grouped_wf[['auc']].quantile(q=[0.25, 0.5, 0.75])

auc
chunk_size               
0.0        0.25  0.978199
           0.50  0.995410
           0.75  1.000000
25.0       0.25  0.994980
           0.50  0.999426
           0.75  1.000000
50.0       0.25  0.993689
           0.50  0.999426
           0.75  1.000000
100.0      0.25  0.990103
           0.50  0.999426
           0.75  1.000000
200.0      0.25  0.985944
           0.50  0.999426
           0.75  1.000000
400.0      0.25  0.983792
           0.50  0.999426
           0.75  1.000000
800.0      0.25  0.988669
           0.50  0.998853
           0.75  1.000000
1600.0     0.25  0.989673
           0.50  0.998853
           0.75  1.000000

In [5]:
wf_aucs = [wf_res.auc[wf_res.chunk_size == cs] for cs in chunk_sizes]
wf_aucs_0 = wf_aucs[0]
print(f"comparing CS=0 to baseline for WF models:", stats.wilcoxon(wf_aucs_0-0.5, alternative='greater'))
for cs_aucs, cs in zip(wf_aucs[1:], chunk_sizes[1:]):
    print(f"comparing CS=0 to CS={cs} for WF models:", stats.wilcoxon(cs_aucs, wf_aucs_0, alternative='greater'))
    es = cs_aucs.to_numpy() / wf_aucs_0.to_numpy()
    print(f"The effect size is: {np.median(es)}")

comparing CS=0 to baseline for WF models: WilcoxonResult(statistic=1275.0, pvalue=3.130562367449191e-10)
comparing CS=0 to CS=25.0 for WF models: WilcoxonResult(statistic=404.0, pvalue=0.07243537740575245)
The effect size is: 1.0
comparing CS=0 to CS=50.0 for WF models: WilcoxonResult(statistic=372.0, pvalue=0.10136595926409131)
The effect size is: 1.0
comparing CS=0 to CS=100.0 for WF models: WilcoxonResult(statistic=354.0, pvalue=0.16700732968271448)
The effect size is: 1.0
comparing CS=0 to CS=200.0 for WF models: WilcoxonResult(statistic=391.0, pvalue=0.1065692060295943)
The effect size is: 1.0
comparing CS=0 to CS=400.0 for WF models: WilcoxonResult(statistic=401.5, pvalue=0.03767750485881228)
The effect size is: 1.0
comparing CS=0 to CS=800.0 for WF models: WilcoxonResult(statistic=458.5, pvalue=0.009370489569644409)
The effect size is: 1.0004361021369004
comparing CS=0 to CS=1600.0 for WF models: WilcoxonResult(statistic=471.5, pvalue=0.00516947579526381)
The effect size is: 1.0

largest effect size was found for 800-spikes chunks, improvement:

In [6]:
# temp here stands for tempporary not temporal
temp = complete[complete.modality == 'morphological']
temp_0 = temp[temp.chunk_size == 0].auc.to_numpy()
temp_800 = temp[temp.chunk_size == 800].auc.to_numpy()
diff = 100 * (temp_800 - temp_0) / temp_0
np.quantile(diff, q=[0.25, 0.5, 0.75])

array([0.        , 0.04361021, 0.38904899])

## spatial

In [7]:
spat_res = complete[complete.modality == 'spatial']
grouped_spat = spat_res.groupby(by=['chunk_size'])
grouped_spat[['auc']].quantile(q=[0.25, 0.5, 0.75])

auc
chunk_size               
0.0        0.25  0.802065
           0.50  0.827883
           0.75  0.848609
25.0       0.25  0.897590
           0.50  0.920539
           0.75  0.933592
50.0       0.25  0.879375
           0.50  0.901320
           0.75  0.920109
100.0      0.25  0.864601
           0.50  0.885255
           0.75  0.906483
200.0      0.25  0.843373
           0.50  0.876363
           0.75  0.904332
400.0      0.25  0.842513
           0.50  0.874068
           0.75  0.897160
800.0      0.25  0.841652
           0.50  0.869191
           0.75  0.891279
1600.0     0.25  0.816695
           0.50  0.859151
           0.75  0.884251

In [8]:
spat_aucs = [spat_res.auc[spat_res.chunk_size == cs] for cs in chunk_sizes]
spat_aucs_0 = spat_aucs[0]
print(f"comparing CS=0 to baseline for Spatial models:", stats.wilcoxon(spat_aucs_0-0.5, alternative='greater'))
for cs_aucs, cs in zip(spat_aucs[1:], chunk_sizes[1:]):
    print(f"comparing CS=0 to CS={cs} for Spatial models:", stats.wilcoxon(cs_aucs, spat_aucs_0, alternative='greater'))
    es = cs_aucs.to_numpy() / spat_aucs_0.to_numpy()
    print(f"The effect size is: {np.median(es)}")

comparing CS=0 to baseline for Spatial models: WilcoxonResult(statistic=1275.0, pvalue=3.7759023791403374e-10)
comparing CS=0 to CS=25.0 for Spatial models: WilcoxonResult(statistic=1275.0, pvalue=3.778464727931783e-10)
The effect size is: 1.0999211595220912
comparing CS=0 to CS=50.0 for Spatial models: WilcoxonResult(statistic=1274.0, pvalue=4.0155454535234565e-10)
The effect size is: 1.0816496697706093
comparing CS=0 to CS=100.0 for Spatial models: WilcoxonResult(statistic=1266.0, pvalue=6.512605320240056e-10)
The effect size is: 1.0679663290066737
comparing CS=0 to CS=200.0 for Spatial models: WilcoxonResult(statistic=1198.5, pvalue=3.055527038421941e-08)
The effect size is: 1.052538290198517
comparing CS=0 to CS=400.0 for Spatial models: WilcoxonResult(statistic=1207.0, pvalue=1.925605797783423e-08)
The effect size is: 1.052501948100291
comparing CS=0 to CS=800.0 for Spatial models: WilcoxonResult(statistic=1152.0, pvalue=3.404098998770698e-07)
The effect size is: 1.059959805653510

largest effect size was found for 25-spikes chunks, improvement:

In [10]:
temp = complete[complete.modality == 'spatial']
temp_0 = temp[temp.chunk_size == 0].auc.to_numpy()
temp_25 = temp[temp.chunk_size == 25].auc.to_numpy()
diff = 100 * (temp_25 - temp_0) / temp_0
print(f"Q25, Q50 and Q75  for chunk size = 25: {np.quantile(diff, q=[0.25, 0.5, 0.75])}")

temp_1600 = temp[temp.chunk_size == 1600].auc.to_numpy()
diff = 100 * (temp_1600 - temp_0) / temp_0
print(f"Q25, Q50 and Q75  for chunk size = 1600: {np.quantile(diff, q=[0.25, 0.5, 0.75])}")

Q25, Q50 and Q75  for chunk size = 25: [ 7.10658387  9.99211595 14.03183484]
Q25, Q50 and Q75  for chunk size = 1600: [-0.9293175   4.15790666  7.06391161]


## Spike-timing

In [11]:
temp_res = complete[complete.modality == 'temporal']
grouped_temp = temp_res.groupby(by=['chunk_size'])
grouped_temp[['auc']].quantile(q=[0.25, 0.5, 0.75])

auc
chunk_size               
0.0        0.25  0.961059
           0.50  0.975043
           0.75  0.986661
25.0       0.25  0.960413
           0.50  0.975330
           0.75  0.982071
50.0       0.25  0.967585
           0.50  0.979633
           0.75  0.985513
100.0      0.25  0.971027
           0.50  0.980207
           0.75  0.986231
200.0      0.25  0.972031
           0.50  0.980207
           0.75  0.989673
400.0      0.25  0.970884
           0.50  0.980780
           0.75  0.989530
800.0      0.25  0.973322
           0.50  0.982501
           0.75  0.990677
1600.0     0.25  0.973752
           0.50  0.981641
           0.75  0.990677

In [12]:
temp_aucs = [temp_res.auc[temp_res.chunk_size == cs] for cs in chunk_sizes]
temp_aucs_0 = temp_aucs[0]
print(f"comparing CS=0 to baseline for Temporal models:", stats.wilcoxon(temp_aucs_0 - 0.5, alternative='greater'))
for cs_aucs, cs in zip(temp_aucs[1:], chunk_sizes[1:]):
    print(f"comparing CS=0 to CS={cs} for Spatial models:", stats.wilcoxon(cs_aucs, temp_aucs_0, alternative='greater'))
    es = cs_aucs.to_numpy() / temp_aucs_0.to_numpy()
    print(f"The effect size is: {np.median(es)}")

comparing CS=0 to baseline for Temporal models: WilcoxonResult(statistic=1275.0, pvalue=3.7750486097697307e-10)
comparing CS=0 to CS=25.0 for Spatial models: WilcoxonResult(statistic=428.0, pvalue=0.9496129118255364)
The effect size is: 0.9967963095116728
comparing CS=0 to CS=50.0 for Spatial models: WilcoxonResult(statistic=619.0, pvalue=0.37525557526793757)
The effect size is: 1.0002901915264073
comparing CS=0 to CS=100.0 for Spatial models: WilcoxonResult(statistic=772.0, pvalue=0.05629903552158046)
The effect size is: 1.0035205085890495
comparing CS=0 to CS=200.0 for Spatial models: WilcoxonResult(statistic=781.0, pvalue=0.04685032388784753)
The effect size is: 1.0029139755694734
comparing CS=0 to CS=400.0 for Spatial models: WilcoxonResult(statistic=909.0, pvalue=0.004382206313944545)
The effect size is: 1.0042733995583841
comparing CS=0 to CS=800.0 for Spatial models: WilcoxonResult(statistic=998.5, pvalue=0.0002461655664405729)
The effect size is: 1.004935629916281
comparing CS=

largest effect size was found for 800-spikes chunks, improvement:

In [13]:
temp = complete[complete.modality == 'temporal']
temp_0 = temp[temp.chunk_size == 0].auc.to_numpy()
temp_800 = temp[temp.chunk_size == 800].auc.to_numpy()
diff = 100 * (temp_800 - temp_0) / temp_0
print(np.quantile(diff, q=[0.25, 0.5, 0.75]))

[-0.16539749  0.49356299  1.29129898]


## Cross-comparisons

In [15]:
temp_800 = complete[complete.chunk_size == 800]
temp_800 = temp_800[temp_800.modality == 'temporal'].auc

wf_800 = complete[complete.chunk_size == 800]
wf_800 = wf_800[wf_800.modality == 'morphological'].auc

spat_25 = complete[complete.chunk_size == 25]
spat_25 = spat_25[spat_25.modality == 'spatial'].auc

print('comparing WF and temporal:', stats.wilcoxon(wf_800, temp_800))
print('comparing WF and spatial:', stats.wilcoxon(wf_800, spat_25))
print('comparing spatial and temporal:', stats.wilcoxon(spat_25, temp_800))

comparing WF and temporal: WilcoxonResult(statistic=109.5, pvalue=3.4471713059458235e-07)
comparing WF and spatial: WilcoxonResult(statistic=0.0, pvalue=7.556929455863566e-10)
comparing spatial and temporal: WilcoxonResult(statistic=0.0, pvalue=7.555220876380673e-10)
